In [1]:
import numpy as np
import pandas as pd
import os
import numpy
import glob
import cv2
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense
print(os.listdir("/mnt/d/deeplearningtut/clg/dlc/exp5/flowers"))
weights = '/mnt/d/deeplearningtut/clg/dlc/exp5/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
print(weights)

2024-09-13 20:12:37.298040: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-13 20:12:37.422497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 20:12:37.477571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 20:12:37.491598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-13 20:12:37.577538: I tensorflow/core/platform/cpu_feature_guar

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
/mnt/d/deeplearningtut/clg/dlc/exp5/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [2]:
#https://stackoverflow.com/questions/49537604/how-to-read-multiple-images-from-multiple-folders-in-python
folders = glob.glob('/mnt/d/deeplearningtut/clg/dlc/exp5/flowers/*')
imagenames_list = []
for folder in folders:
    for f in glob.glob(folder+'/*.jpg'):
        imagenames_list.append(f)


In [3]:
def label_img(image):
    word_label = image.split('/')[4]
    if word_label == 'daisy':
        return [1,0,0,0,0]
    elif word_label == 'dandelion':
        return [0,1,0,0,0]
    elif word_label == 'rose':
        return [0,0,1,0,0]
    elif word_label == 'tulip':
        return [0,0,0,1,0]
    else:
        return [0,0,0,0,1]

In [4]:
train = []        

for image in imagenames_list:
    label = label_img(image)
    train.append([np.array(cv2.resize(cv2.imread(image),(224,224))), np.array(label)])
    np.random.shuffle(train)

In [5]:
X = np.array([i[0] for i in train])
X = X/255
Y = np.array([i[1] for i in train])

print(X.shape)
print(Y.shape)


(4317, 224, 224, 3)
(4317, 5)


In [6]:
import os
print(os.path.exists('/mnt/d/deeplearningtut/clg/dlc/exp5/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'))
print(os.path.exists("/mnt/d/deeplearningtut/clg/dlc/exp5/flowers"))


True
True


In [7]:
model = Sequential()
model.add(VGG16(include_top=False, weights=weights, input_shape=(224,224,3)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


I0000 00:00:1726238581.656374   17030 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726238581.751724   17030 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726238581.751771   17030 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726238581.756360   17030 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726238581.756418   17030 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,562,821 (105.14 MB)

 Trainable params: 27,562,821 (105.14 MB)

 Non-trainable params: 0 (0.00 B)

: 

In [9]:
Fit = model.fit(X, Y, epochs = 1, validation_split = 0.30,batch_size=16)

: 

: 